In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import os, json
import matplotlib.pyplot as plt 
import matplotlib as mpl
%matplotlib inline 
import seaborn as sns
import scipy.stats as stats
from scipy.stats import norm
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier

from sklearn.tree import export_graphviz 
import graphviz

from sklearn.metrics import confusion_matrix
from os import path
from PIL import Image
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
import pickle

In [2]:
data=pd.read_csv('data/bdata.csv') 
data=data.drop(['Unnamed: 0'],axis=1)


cats=list(data['category.slug'].unique())
print('\033[1m'+'Optioanl categories:'+'\033[0m',cats)

cat_select=''
while cat_select not in cats:
    cat_select=input('\033[1m'+'Select your category: '+'\033[0m')
else:
        data=data.loc[data['category.slug']==cat_select]
        print(data['category.slug'][:1])
        print ('\033[1m'+'Category is OK'+'\033[0m')
name=input ('\033[1m'+'Enter your campaign name: '+'\033[0m')
blurb=input ('\033[1m'+'Enter your campaign description: '+'\033[0m')
print('\n')
print('\033[1m'+'Project category: '+'\033[0m',cat_select)
print('\033[1m'+'Project Name: '+'\033[0m',name)
print('\033[1m'+'Project Description: '+'\033[0m',blurb)

Optioanl categories: ['food', 'art', 'publishing', 'comics', 'journalism', 'crafts', 'design', 'film & video', 'dance', 'photography', 'technology', 'music', 'games', 'fashion', 'theater']
Select your category: crafts
16    crafts
Name: category.slug, dtype: object
Category is OK
Enter your campaign name: Design a new Table
Enter your campaign description: Fantastic new table design very trendy


Project category:  crafts
Project Name:  Design a new Table
Project Description:  Fantastic new table design very trendy


# Name and Description improvement

In [3]:
import collections

words = data[data.state == 'successful']['name']
words=words.values
import re
words=words.tolist()
words[1]
words=str(words)
num=[int(s) for s in words.split() if s.isdigit()]

words = data[data.state == 'successful']['name']
words=words.apply(lambda x: x.lower())
words=list(filter(lambda x:x, map(lambda x:re.sub(r'[^A-Za-z]', ' ', x), words)))

words=str(words).split(" ")
words=list(filter(lambda x:x, map(lambda x:re.sub(r'[^A-Za-z]',"", x), words)))

words.append([num])

words= str(words)
words=words.replace(' ','')
words=words.replace("'",'')
words=words.split(sep=',')

s_counter = collections.Counter(words)
a=s_counter.most_common(n=150)
dfgwords=pd.DataFrame(a,columns=['word','freq'])
a = [x[0] for x in a]
dfgwords

words = data[data.state == 'failed']['name']
words=words.values
import re
words=words.tolist()
words[1]
words=str(words)
num=[int(s) for s in words.split() if s.isdigit()]

words = data[data.state == 'failed']['name']
words=words.apply(lambda x: x.lower())
words=list(filter(lambda x:x, map(lambda x:re.sub(r'[^A-Za-z]', ' ', x), words)))

words=str(words).split(" ")
words=list(filter(lambda x:x, map(lambda x:re.sub(r'[^A-Za-z]',"", x), words)))

words.append([num])

words= str(words)
words=words.replace(' ','')
words=words.replace("'",'')
words=words.split(sep=',')

f_counter = collections.Counter(words)
b=f_counter.most_common(n=150)
dfbwords=pd.DataFrame(b,columns=['word','freq'])
b = [x[0] for x in b]


#just the uniqe words
s_words=list(set(a) - set(b))
f_words=list(set(b) - set(a))
wordsT=pd.DataFrame()
wordsT['suc_w']=list(set(a) - set(b))
wordsT['fail_w']=list(set(b) - set(a))
wordsT['c']=1
print('Good words for name:', s_words)
print()
print('Bad words for name:',f_words)

def show_wordcloud(wordsT, title = None):
    '''Split names by space and generate word counts.'''
    wordcloud = WordCloud(
        background_color='white',
        max_words=150,
        max_font_size=40, 
        scale=3,
        random_state=1 
    ).generate(str(wordsT))

    fig = plt.figure(1, figsize=(12, 12))
    plt.axis('off')
    if title: 
        fig.suptitle(title, fontsize=20)
        fig.subplots_adjust(top=2.3)

    plt.imshow(wordcloud)
    plt.show()

#show_wordcloud(wordsT[wordsT.c == 1]['suc_w'])
#show_wordcloud(wordsT[wordsT.c == 1]['fail_w'])

Good words for name: ['david', 'magnetic', 'bowlmaker', 'eagle', 'good', 'whiskey', 'inspired', 'ltd', 'press', 'yarns', 'oils', 'walsh', 'quilt', 'razors', 'wick', 'n', 'oil', 'book', 'limited', 'fine', 'ceramic', 'pattern', 'exotic', 'holiday', 'accessories', 'bowls', 'cat', 'stationery', 'fire', 'turned', 'oak', 'bourbon', 'veterans', 'knitted', 'barrel', 'kids', 'trees', 'scottish', 'kiln', 'games', 'expansion', 'sustainable', 'leather', 'dyed', 'pen', 'ceramics', 'card', 'fiber', 'day', 'little', 'system', 'edition', 'real', 'black', 'ancient', 'writing']

Bad words for name: ['need', 'old', 'd', 'at', 'get', 'free', 'business', 'dog', 'skin', 'pet', 'guitars', 'cutting', 'our', 'personalized', 'suspended', 't', 'beauty', 'store', 'rustic', 'soaps', 'soap', 'bath', 'can', 'tree', 'friendly', 'decor', 'time', 'dream', 'fun', 'stand', 'embroidery', 'crafting', 'bottle', 'laser', 'boutique', 'scented', 'tiny', 'reclaimed', 'american', 'items', 'scents', 'm', 'wax', 'fund', 'jewelry',

In [4]:
words = data[data.state == 'successful']['blurb']
words=words.values
import re
words=words.tolist()
words[1]
words=str(words)
num=[int(s) for s in words.split() if s.isdigit()]

words = data[data.state == 'successful']['blurb']
words=words.apply(lambda x: x.lower())
words=list(filter(lambda x:x, map(lambda x:re.sub(r'[^A-Za-z]', ' ', x), words)))

words=str(words).split(" ")
words=list(filter(lambda x:x, map(lambda x:re.sub(r'[^A-Za-z]',"", x), words)))

words.append([num])

words= str(words)
words=words.replace(' ','')
words=words.replace("'",'')
words=words.split(sep=',')

s_counter = collections.Counter(words)
a=s_counter.most_common(n=200)
dfgblurb=pd.DataFrame(a,columns=['word','freq'])
a = [x[0] for x in a]
a

words = data[data.state == 'failed']['blurb']
words=words.values
import re
words=words.tolist()
words[1]
words=str(words)
num=[int(s) for s in words.split() if s.isdigit()]

words = data[data.state == 'failed']['blurb']
words=words.apply(lambda x: x.lower())
words=list(filter(lambda x:x, map(lambda x:re.sub(r'[^A-Za-z]', ' ', x), words)))

words=str(words).split(" ")
words=list(filter(lambda x:x, map(lambda x:re.sub(r'[^A-Za-z]',"", x), words)))

words.append([num])

words= str(words)
words=words.replace(' ','')
words=words.replace("'",'')
words=words.split(sep=',')

b_counter = collections.Counter(words)
b=b_counter.most_common(n=200)
dfbblurb=pd.DataFrame(b,columns=['word','freq'])
b = [x[0] for x in b]
b

#just the uniqe words
s_blurb=list(set(a) - set(b))
f_blurb=list(set(b) - set(a))
wordsB=pd.DataFrame()
wordsB['suc_b']=list(set(a) - set(b))
wordsB['fail_b']=list(set(b) - set(a))
wordsB['c']=1
print('Good words for description:',s_blurb)
print()
print('Bad words for description:',f_blurb)

def show_wordcloud(wordsB, title = None):
    '''Split names by space and generate word counts.'''
    wordcloud = WordCloud(
        background_color='white',
        max_words=100,
        max_font_size=40, 
        scale=3,
        random_state=1 # chosen at random by flipping a coin it was heads
    ).generate(str(wordsB))

    fig = plt.figure(1, figsize=(12, 12))
    plt.axis('off')
    if title: 
        fig.suptitle(title, fontsize=20)
        fig.subplots_adjust(top=2.3)

    plt.imshow(wordcloud)
    plt.show()
#show_wordcloud(wordsB[wordsB.c == 1]['suc_b'])
#show_wordcloud(wordsB[wordsB.c == 1]['fail_b'])

Good words for description: ['two', 'based', 'kickstarter', 'set', 'pottery', 'good', 'better', 'space', 'most', 'beard', 'favorite', 'patterns', 'domestic', 'book', 'working', 'limited', 'yarn', 'fine', 'her', 'through', 'raise', 'exotic', 'studio', 'support', 'collection', 'bowls', 'raising', 'perfect', 'paper', 'turned', 'brand', 'years', 'greeting', 'friends', 'barrels', 'woods', 'range', 'kiln', 'reclaimed', 'women', 'year', 'christmas', 'knitting', 'sustainable', 'dyed', 'turn', 'pen', 'fund', 'card', 'fiber', 'little', 'grow', 'funds', 'keep', 'woodworking']

Bad words for description: ['crafts', 'family', 'people', 'baby', 'projects', 'well', 'give', 'many', 'everyone', 'provide', 'skin', 'open', 'those', 'crochet', 'store', 'about', 'created', 'come', 'been', 'only', 'starting', 'r', 'soaps', 'pieces', 'different', 'bath', 'look', 'gift', 'friendly', 'decor', 'back', 'dream', 'not', 'supplies', 'where', 'scented', 'trying', 'kids', 'sell', 'company', 'machine', 'started', 'boa

In [5]:
def goodblurb(s):
    s = str(s)
    s = re.sub(r'[^A-Za-z]', ' ', s)
    s= s.lower()
    s= s.split()
    a=0
    for i in s_blurb:
        if i in s:
            a+=1
    return (a)

data['goodblurb'] = data.blurb.apply(goodblurb)
def badblurb(s):
    s = str(s)
    s = re.sub(r'[^A-Za-z]', ' ', s)
    s= s.lower()
    s= s.split()
    a=0
    for i in f_blurb:
        if i in s:
            a+=1
    return (a)

data['badblurb'] = data.blurb.apply(badblurb)


In [6]:
def goodname(s):
    s = str(s)
    s = re.sub(r'[^A-Za-z]', ' ', s)
    s= s.lower()
    s= s.split()
    a=0
    for i in s_words:
        if i in s:
            a+=1
    return (a)

data['goodname'] = data.name.apply(goodname)

def badname(s):
    s = str(s)
    s = re.sub(r'[^A-Za-z]', ' ', s)
    s= s.lower()
    s= s.split()
    a=0
    for i in f_words:
        if i in s:
            a+=1
    return (a)

data['badname'] = data.name.apply(badname)


In [7]:
data['goodname_mean']=data['goodname'].mean()
data['goodname_corr']=data['goodname'].corr(data['state_num'])
data['badname_mean']=data['badname'].mean()
data['badname_corr']=data['badname'].corr(data['state_num'])
data['goodblurb_mean']=data['goodblurb'].mean()
data['goodblurb_corr']=data['goodblurb'].corr(data['state_num'])
data['badblurb_mean']=data['badblurb'].mean()
data['badblurb_corr']=data['badblurb'].corr(data['state_num'])

data['goodname_score']=((data['goodname']-data['goodname_mean'])/data['goodname_mean'])*data['goodname_corr']
data['badname_score']=((data['badname']-data['badname_mean'])/data['badname_mean'])*data['badname_corr']
data['goodblurb_score']=((data['goodblurb']-data['goodblurb_mean'])/data['goodblurb_mean'])*data['goodblurb_corr']
data['badblurb_score']=((data['badblurb']-data['badblurb_mean'])/data['badblurb_mean'])*data['badblurb_corr']


In [8]:
pickle.dump(data, open("odata.dat", "wb")) # wb = write binary

In [9]:
data = pickle.load(open("odata.dat", "rb"))

In [10]:
word=data[['state_num','name_len','name_is_question',
           'name_is_exclamation','name_is_upper','name_non_character',
           'name_number_of_word','name_vowel_ratio','blurb_number_of_word',
           'blurb_vowel_ratio','blurb_non_character','goodname','badname','goodblurb','badblurb']]
corr_word=word.corr()
corr_word=corr_word.loc['state_num']
corr_word.nlargest(17)

corr_word_re=corr_word.values.reshape(1,15)

corr_t=pd.DataFrame(data=corr_word_re,columns=corr_word.index)
corr_t.rename(columns=lambda x: x+'_corr', inplace=True)


In [11]:
word_m=word
word_m=word_m.mean()
corr_mean_re=word_m.values.reshape(1,15)

corr_mean_re=pd.DataFrame(data=corr_mean_re,columns=word_m.index)
corr_mean_re.rename(columns=lambda x: x+'_mean', inplace=True)
 


In [12]:
data=pd.DataFrame()
data['name']=[name]
data['blurb']=[blurb]

In [13]:
pickle.dump(data, open("wdata.dat", "wb")) # wb = write binary

In [14]:
data = pickle.load(open("wdata.dat", "rb"))

In [15]:
# length of the name
data['name_len'] = data.name.str.len()
# if name contains a question mark
data['name_is_question'] = (data.name.str[-1] == '?').astype(int)
# if name contains an exclamation mark
data['name_is_exclamation'] = (data.name.str[-1] == '!').astype(int)
# if name is uppercase
data['name_is_upper'] = data.name.str.isupper().astype(float)
def count_non_character(row):
    '''Number of non character in the sentence'''
    return sum((0 if c.isalpha() else 1 for c in str(row)))
# number of non character in the name
data['name_non_character'] = data.name.apply(count_non_character)
# number of words in the name
data['name_number_of_word'] = data.name.apply(lambda x: len(str(x).split(' ')))
# We generate new feature based on ratio between vowels and other alpha characters
def countVowelstoLettersRatio(s):
    '''Count ratio between vowels and letters'''
    s = str(s)
    count = 1  
    vowels = 0
    for i in s:
        if i.isalpha():
            count = count + 1
            if i in 'aeiou':
                vowels = vowels + 1
    return ((vowels * 1.0) / count)

# for each name calculate vowels ratio
data['name_vowel_ratio'] = data.name.apply(countVowelstoLettersRatio)

#blurb
data['blurb_number_of_word'] = data.blurb.apply(lambda x: len(str(x).split(' ')))
data['blurb_vowel_ratio'] = data.blurb.apply(lambda x: len(str(x).split(' ')))
data['blurb_non_character'] = data.blurb.apply(count_non_character)


In [16]:
data=data.join(corr_t)
data=data.join(corr_mean_re)


In [17]:
blurb=re.sub(r'[^A-Za-z]', ' ', blurb)
blurb=blurb.lower()
blurb = blurb.split()

gb=[]
for i in s_blurb:
    if i in blurb:
        gb.append(i)
bb=[]
for i in f_blurb:
    if i in blurb:
        bb.append(i)
ngb=len(gb)
nbb=len(bb)
data['goodblurb']=ngb
data['badblurb']=nbb
print (blurb)
print('Good words in campaign description',gb,ngb)
print('Bad words in campaign description',bb,nbb)

['fantastic', 'new', 'table', 'design', 'very', 'trendy']
Good words in campaign description [] 0
Bad words in campaign description [] 0


In [18]:
name=re.sub(r'[^A-Za-z]', ' ', name)
name=name.lower()
name = name.split()

gn=[]
for i in s_words:
    if i in name:
        gn.append(i)
bn=[]
for i in f_words:
    if i in name:
        bn.append(i)
ngn=len(gn)
nbn=len(bn)
data['goodname']=ngn
data['badname']=nbn

print (name)
print('Good words in campaign name',gn,ngn)
print('Bad words campaign name',bn,nbn)

['design', 'a', 'new', 'table']
Good words in campaign name [] 0
Bad words campaign name [] 0


In [19]:
score=pd.DataFrame()
score['name_len']=((data['name_len']-data['name_len_mean'])/-data['name_len_mean'])*data['name_len_corr']
score['name_is_question']=((data['name_is_question']-data['name_is_question_mean'])/data['name_is_question_mean'])*data['name_is_question_corr']
score['name_is_exclamation']=((data['name_is_exclamation']-data['name_is_exclamation_mean'])/data['name_is_exclamation_mean'])*data['name_is_exclamation_corr']
score['name_is_upper']=((data['name_is_upper']-data['name_is_upper_mean'])/data['name_is_upper_mean'])*data['name_is_upper_corr']
score['name_non_character']=((data['name_non_character']-data['name_non_character_mean'])/data['name_non_character_mean'])*data['name_non_character_corr']
score['name_number_of_word']=((data['name_len']-data['name_len_mean'])/data['name_len_mean'])*data['name_len_corr']
score['name_vowel_ratio']=((data['name_vowel_ratio']-data['name_vowel_ratio_mean'])/data['name_vowel_ratio_mean'])*data['name_vowel_ratio_corr']
score['blurb_vowel_ratio']=((data['blurb_vowel_ratio']-data['blurb_vowel_ratio_mean'])/data['blurb_vowel_ratio_mean'])*data['blurb_vowel_ratio_corr']
score['blurb_non_character']=((data['blurb_non_character']-data['blurb_non_character_mean'])/data['blurb_non_character_mean'])*data['blurb_non_character_corr']
score['goodname']=((data['goodname']-data['goodname_mean'])/data['goodname_mean'])*data['goodname_corr']
score['badname']=((data['badname']-data['badname_mean'])/data['badname_mean'])*data['badname_corr']
score['goodblurb']=((data['goodblurb']-data['goodblurb_mean'])/data['goodblurb_mean'])*data['goodblurb_corr']
score['badblurb']=((data['badblurb']-data['badblurb_mean'])/data['badblurb_mean'])*data['badblurb_corr']
score['Total_Score'] = score[score.columns].sum(axis=1)

# Scores

In [20]:
print('\033[1m' +'NAME improvments:'+'\033[0m')
print('\033[1m' +'GOOD words in this campaign NAME:'+'\033[0m',gn,ngn)
print('\033[1m' +'BAD words in this campaign NAME:'+'\033[0m',bn,nbn)
print('\033[1m' +'GOOD words for NAME in this category are:'+'\033[0m', s_words)

print('\n')
print('\033[1m' +'DESCRIPTION improvments:'+'\033[0m')
print('\033[1m' +'GOOD words in this campaign DESCRIPTION:'+'\033[0m',gb,ngb)
print('\033[1m' +'BAD words in this campaign DESCRIPTION:'+'\033[0m',bb,nbb)
print('\033[1m' +'GOOD words for DESCRIPTION in this category are:'+'\033[0m', s_blurb)

print('\n')
print('\033[1m' +'Table Of Scores'+'\033[0m')
print(score.T)
print('\n')
print('\033[1m' +'Total Score is:'+'\033[0m', score['Total_Score'][0])

NAME improvments:
GOOD words in this campaign NAME: [] 0
BAD words in this campaign NAME: [] 0
GOOD words for NAME in this category are: ['david', 'magnetic', 'bowlmaker', 'eagle', 'good', 'whiskey', 'inspired', 'ltd', 'press', 'yarns', 'oils', 'walsh', 'quilt', 'razors', 'wick', 'n', 'oil', 'book', 'limited', 'fine', 'ceramic', 'pattern', 'exotic', 'holiday', 'accessories', 'bowls', 'cat', 'stationery', 'fire', 'turned', 'oak', 'bourbon', 'veterans', 'knitted', 'barrel', 'kids', 'trees', 'scottish', 'kiln', 'games', 'expansion', 'sustainable', 'leather', 'dyed', 'pen', 'ceramics', 'card', 'fiber', 'day', 'little', 'system', 'edition', 'real', 'black', 'ancient', 'writing']


DESCRIPTION improvments:
GOOD words in this campaign DESCRIPTION: [] 0
BAD words in this campaign DESCRIPTION: [] 0
GOOD words for DESCRIPTION in this category are: ['two', 'based', 'kickstarter', 'set', 'pottery', 'good', 'better', 'space', 'most', 'beard', 'favorite', 'patterns', 'domestic', 'book', 'working', '